In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore


from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

import os
import re

from dotenv import load_dotenv
load_dotenv()

COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

In [2]:
def get_pdf_file_names(folder_path):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.

    Args:
        folder_path (str): PDF 파일이 있는 폴더의 경로.

    Returns:
        list: PDF 파일 이름들의 리스트.
    """
    try:
        # 폴더 내 파일 리스트 가져오기
        all_files = os.listdir(folder_path)
        
        # 확장자가 '.pdf'인 파일만 필터링
        pdf_files = [file.replace(".pdf","") for file in all_files if file.lower().endswith('.pdf')]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

In [3]:
def load_and_split_tax_law(file_path,repeat_pattern):
    """
    PDF 파일을 로드하고, 반복 텍스트 제거, 조항 기준으로 분할 및 연결 후 임베딩 준비.
    Args:
        file_path (str): PDF 파일 경로.
    Returns:
        list: 연결된 텍스트 조각들의 리스트.
    """
    import re
    from langchain.document_loaders import PyMuPDFLoader
    from langchain_openai import OpenAIEmbeddings

    # PDF 로드
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()
    
    # 전체 텍스트 가져오기
    full_text = " ".join([doc.page_content for doc in documents])

    # 1. 반복 텍스트 제거
    full_text = re.sub(repeat_pattern, "", full_text)

    # 2. '제n조'를 기준으로 분할
    split_pattern = r"\n(제\d+조[^\s]*)"
    chunks = re.split(split_pattern, full_text)

    # 3. '제n조'와 연결하여 완전한 문단 구성 + 부칙 처리
    connected_chunks = []
    current_chunk = ""
    is_buchik_section = False  # 부칙 여부 플래그

    for chunk in chunks:
        if re.search(r"\n\s*<부칙>", chunk):  # 부칙 시작 여부 확인
            is_buchik_section = True

        if chunk.startswith("제") and "조" in chunk:  # 새로운 조항 시작
            if is_buchik_section:  # 부칙 섹션이라면 접두사 추가
                chunk = "부칙-" + chunk

            if current_chunk:  # 이전 조항 저장
                connected_chunks.append(current_chunk.strip())
            current_chunk = chunk  # 새 조항으로 시작
        else:
            current_chunk += f" {chunk}"  # 기존 조항에 내용 추가

    if current_chunk:
        connected_chunks.append(current_chunk.strip())


    return connected_chunks, full_text



In [4]:
def get_vector_store(pdf_files):
    """
    PDF 파일들을 처리하여 임베딩을 Chroma Vector Store에 저장합니다.

    Args:
        pdf_files (list): PDF 파일 이름들의 리스트.
        vector_store_path (str): Chroma Vector Store를 저장할 경로.
    """
    from langchain_chroma import Chroma
    from langchain_core.documents import Document
    
    # 각 PDF 파일에 대해 임베딩 처리
    # for file in pdf_files:
    all_docs = []
    
    for i in range(3):
        file = pdf_files[i]
        path = f"data/tax_law/{file}.pdf"  # PDF 파일 경로
        repeat_pattern = rf"법제처\s*\d+\s*국가법령정보센터\n{file.replace('_', ' ')}\n" 
        chunks, full_text = load_and_split_tax_law(path, repeat_pattern)
        

        for idx, chunk in enumerate(chunks):
            _doc = Document(metadata={"title": file, "full_text": full_text[idx], }, page_content=chunk),
            all_docs.extend(_doc)

    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
    
    vector_store = Chroma.from_documents(
        documents=all_docs,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )
    
    return vector_store
        
folder_path = "data/tax_law" 
pdf_files = get_pdf_file_names(folder_path)
vector_store = get_vector_store(pdf_files)

In [5]:
vector_store._collection.count()

134

In [6]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":10}
)


In [7]:
# Prompt Template 생성
messages = [
        ("ai", """
        You are a helpful assistant. Answer question using only the following context. 
        If you don't know the answer, just say you don't know. Don't make it up. 
        Answer in Korean.
        
        {context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser



In [10]:
chain.invoke("개별소비세법이 뭐야?")


'개별소비세법은 특정 물품, 입장, 유흥 음식행위 및 영업행위에 대해 부과되는 세금을 규정하는 법률입니다. 이 법에 따라 과세 시기는 물품이 제조장에서 반출되거나 수입신고를 할 때, 과세장소에 입장할 때, 유흥음식행위를 할 때, 그리고 과세영업장소에서 영업행위를 할 때로 정해져 있습니다. 또한, 외교관 면세와 같은 특정 경우에는 개별소비세가 면제될 수 있는 조항도 포함되어 있습니다.'